In [2]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

In [17]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz")
predictorOutput = predictor.predict(
document="joe is cool. he is my friend.")

In [18]:
print(predictorOutput)
print(predictorOutput["clusters"])
clusters = predictorOutput["clusters"]


{'top_spans': [[0, 0], [4, 4], [6, 6]], 'antecedent_indices': [[0, 1, 2], [0, 1, 2], [0, 1, 2]], 'predicted_antecedents': [-1, 0, -1], 'document': ['joe', 'is', 'cool', '.', 'he', 'is', 'my', 'friend', '.'], 'clusters': [[[0, 0], [4, 4]]]}
[[[0, 0], [4, 4]]]


In [7]:
# all_clusters = dict()
# for cluster in clusters:
#     for words in cluster:
        
#         for w in words:
#             #print(w)
#             print(predictorOutput["document"][w])
            

Paul
Paul
He
He


In [20]:
def generateClusters(predictorOutput):
    clusters = predictorOutput['clusters']
    document = predictorOutput['document']
    document = predictorOutput['document']
    n = 0
    doc = {}
    for obj in document:
        doc.update({n :  obj}) #what I'm doing here is creating a dictionary of each word with its respective index, making it easier later.
        n = n+1

    clus_all = []
    cluster = []
    clus_one = {}
    for i in range(0, len(clusters)):
        one_cl = clusters[i]
        for count in range(0, len(one_cl)):
            obj = one_cl[count]
            for num in range((obj[0]), (obj[1]+1)):
                for n in doc:
                    if num == n:
                        cluster.append(doc[n])
        clus_all.append(cluster)
        cluster = [] 

    print(clus_all) #shows all coreferences
    return clus_all

In [21]:
# generateReferences takes in a context (1+ previous sentences)
# and a current sentence and replaces all broad references with  
# coreferences and the broad reference itself
# TODO: would we like to create a list of all "broad references"
broad_refs = ["he", "it", "she"]
def generateReferences(context, curr_sentence):
    clus_all = generateClusters(predictorOutput)
    for cluster in clus_all:
        for word in cluster:
                print(word)
                if word in broad_refs:
                    results = [curr_sentence.replace(word, newWord) for newWord in cluster]
                    print(results)
                    
# example: Paul is cool. He is cool. 
generateReferences("joe is cool. he is my friend", "he is cool.")

    

[['joe', 'he']]
joe
he
['joe is cool.', 'he is cool.']
